In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.utils import get_file


path = get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')
text = open(path, 'rb').read().decode(encoding='utf-8')


chars = sorted(set(text))
char_to_index = {char: i for i, char in enumerate(chars)}
index_to_char = {i: char for i, char in enumerate(chars)}
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])


x = np.zeros((len(sentences), maxlen, len(chars)), dtype=bool)
y = np.zeros((len(sentences), len(chars)), dtype=bool)

for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_to_index[char]] = 1
    y[i, char_to_index[next_chars[i]]] = 1


model = Sequential()
model.add(SimpleRNN(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))


model.compile(loss='categorical_crossentropy', optimizer=RMSprop(learning_rate=0.01))


history = model.fit(x, y, batch_size=128, epochs=20)


def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


generated_text = "ROMEO: "
for i in range(400):
    sampled = np.zeros((1, maxlen, len(chars)))
    for t, char in enumerate(generated_text[-maxlen:]):
        sampled[0, t, char_to_index[char]] = 1.

    preds = model.predict(sampled, verbose=0)[0]
    next_index = sample(preds, temperature=0.5)
    next_char = index_to_char[next_index]
    generated_text += next_char

print(generated_text)


1115394/1115394 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/20
2905/2905 ━━━━━━━━━━━━━━━━━━━━ 123s 42ms/step - loss: 3.1256
Epoch 2/20
2905/2905 ━━━━━━━━━━━━━━━━━━━━ 113s 39ms/step - loss: 2.4862
Epoch 3/20
2905/2905 ━━━━━━━━━━━━━━━━━━━━ 112s 39ms/step - loss: 2.6125
Epoch 4/20
2905/2905 ━━━━━━━━━━━━━━━━━━━━ 147s 40ms/step - loss: 2.9417
Epoch 5/20
2905/2905 ━━━━━━━━━━━━━━━━━━━━ 117s 40ms/step - loss: 2.6447
Epoch 6/20
2905/2905 ━━━━━━━━━━━━━━━━━━━━ 140s 40ms/step - loss: 2.5116
Epoch 7/20
2905/2905 ━━━━━━━━━━━━━━━━━━━━ 140s 39ms/step - loss: 2.4759
Epoch 8/20
2905/2905 ━━━━━━━━━━━━━━━━━━━━ 149s 42ms/step - loss: 2.4543
Epoch 9/20
2905/2905 ━━━━━━━━━━━━━━━━━━━━ 159s 47ms/step - loss: 2.4339
Epoch 10/20
2905/2905 ━━━━━━━━━━━━━━━━━━━━ 137s 46ms/step - loss: 2.4338
Epoch 11/20
2905/2905 ━━━━━━━━━━━━━━━━━━━━ 115s 39ms/step - loss: 2.4338
Epoch 12/20
2905/2905 ━━━━━━━━━━━━━━━━━━━━ 116s 40ms/step - loss: 2.4549
Epoch 13/20
2905/2905 ━━━━━━━━━━━━━━━━━━━━ 141s 40ms/step - loss: 2.4526
Epoch 14/20
2905/2905 ━━━━━━━━━━━━━━━━━━━━ 144s 40ms/step - 

<ipython-input-1-1475c2bf2469>:48: RuntimeWarning: divide by zero encountered in log
  preds = np.log(preds) / temperature


ROMEO: ee,eoeeii eeeeeeeeeeieeeee :  AEELA LARK:
And be my meied butbom my me my me bempe beame mint be my be beme bemp ben my beate biome bine be my bigh be be bime bime be my me be me mye bembint be be beme ble mye my ben bite be be my mhol be be bemone bir be my my bemblo wom bithe be bemere my my peame be beine be be my me bin be be be bin be be bame bemm be my my bemmine me my my be me bemme by my a


In [ ]:
# Bi-Directional RNN Model


bi_rnn_model = Sequential()
bi_rnn_model.add(tf.keras.layers.Bidirectional(SimpleRNN(128), input_shape=(maxlen, len(chars))))
bi_rnn_model.add(Dense(len(chars), activation='softmax'))

bi_rnn_model.compile(loss='categorical_crossentropy', optimizer=RMSprop(learning_rate=0.01))

bi_rnn_history = bi_rnn_model.fit(x, y, batch_size=128, epochs=20)

generated_text_bi_rnn = "ROMEO: "
for i in range(400):
    sampled = np.zeros((1, maxlen, len(chars)))
    for t, char in enumerate(generated_text_bi_rnn[-maxlen:]):
        sampled[0, t, char_to_index[char]] = 1.

    preds = bi_rnn_model.predict(sampled, verbose=0)[0]
    next_index = sample(preds, temperature=0.5)
    next_char = index_to_char[next_index]
    generated_text_bi_rnn += next_char

print("Generated text from Bi-Directional RNN:\n")
print(generated_text_bi_rnn)


Epoch 1/20
2905/2905 ━━━━━━━━━━━━━━━━━━━━ 216s 73ms/step - loss: 3.4607
Epoch 2/20
2905/2905 ━━━━━━━━━━━━━━━━━━━━ 258s 72ms/step - loss: 3.2970
Epoch 3/20
2905/2905 ━━━━━━━━━━━━━━━━━━━━ 210s 72ms/step - loss: 3.1604
Epoch 4/20
2905/2905 ━━━━━━━━━━━━━━━━━━━━ 259s 71ms/step - loss: 3.0179
Epoch 5/20
2905/2905 ━━━━━━━━━━━━━━━━━━━━ 263s 72ms/step - loss: 2.9090
Epoch 6/20
2905/2905 ━━━━━━━━━━━━━━━━━━━━ 255s 69ms/step - loss: 2.8888
Epoch 7/20
2905/2905 ━━━━━━━━━━━━━━━━━━━━ 203s 70ms/step - loss: 2.8961
Epoch 8/20
2905/2905 ━━━━━━━━━━━━━━━━━━━━ 205s 71ms/step - loss: 2.8567
Epoch 9/20
2905/2905 ━━━━━━━━━━━━━━━━━━━━ 263s 71ms/step - loss: 2.8372
Epoch 10/20
2905/2905 ━━━━━━━━━━━━━━━━━━━━ 260s 70ms/step - loss: 2.8579
Epoch 11/20
2905/2905 ━━━━━━━━━━━━━━━━━━━━ 269s 73ms/step - loss: 2.8785
Epoch 12/20
2905/2905 ━━━━━━━━━━━━━━━━━━━━ 273s 77ms/step - loss: 2.8903
Epoch 13/20
2905/2905 ━━━━━━━━━━━━━━━━━━━━ 215s 74ms/step - loss: 2.9023
Epoch 14/20
2905/2905 ━━━━━━━━━━━━━━━━━━━━ 210s 72ms/step - 

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, SimpleRNN, Dense, Flatten
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.utils import get_file

path = get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')
text = open(path, 'rb').read().decode(encoding='utf-8')

chars = sorted(set(text))
char_to_index = {char: i for i, char in enumerate(chars)}
index_to_char = {i: char for i, char in enumerate(chars)}
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])

x = np.zeros((len(sentences), maxlen, len(chars)), dtype=bool)
y = np.zeros((len(sentences), len(chars)), dtype=bool)

for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_to_index[char]] = 1
    y[i, char_to_index[next_chars[i]]] = 1

hybrid_model = Sequential()

hybrid_model.add(Conv1D(64, 5, activation='relu', input_shape=(maxlen, len(chars))))
hybrid_model.add(MaxPooling1D(2))

hybrid_model.add(SimpleRNN(128))


hybrid_model.add(Dense(len(chars), activation='softmax'))

hybrid_model.compile(loss='categorical_crossentropy', optimizer=RMSprop(learning_rate=0.01))

hybrid_history = hybrid_model.fit(x, y, batch_size=128, epochs=20)

def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

generated_text_hybrid = "ROMEO: "
for i in range(400):
    sampled = np.zeros((1, maxlen, len(chars)))
    for t, char in enumerate(generated_text_hybrid[-maxlen:]):
        sampled[0, t, char_to_index[char]] = 1.

    preds = hybrid_model.predict(sampled, verbose=0)[0]
    next_index = sample(preds, temperature=0.5)
    next_char = index_to_char[next_index]
    generated_text_hybrid += next_char

print("Generated text from Hybrid CNN + RNN:\n")
print(generated_text_hybrid)


Epoch 1/20
 142/2905 ━━━━━━━━━━━━━━━━━━━━ 1:53 41ms/step - loss: 3.7627